In [ ]:
from __future__ import annotations
from enum import Enum
import hashlib

class Status(Enum):   # 노드의 상태를 열거형으로 정의
    OCCUPIED = 0    # 데이터 저장
    EMPTY = 1    # 비어있음
    DELETED = 2    # 삭제 완료
    
class Bucket:
    
    def __init__(self, key = None, value = None, stat = Status.EMPTY):    # 매개변수 기본값 None
        self.key = key
        self.value = value
        self.stat = stat
        
    def set(self, key, value, stat):
        self.key = key
        self.value = value
        self.stat = stat
            
    def set_status(self, stat):
        self.stat = stat
            
class OpenHash:
    
    def __init__(self, capacity):
        self.capacity = capacity
        self.table = [Bucket()] * self.capacity
        
    def hash_value(self, key):
        if isinstance(key, int):
            return key % self.capacity
        return(int(hashlib.md5(str(key).encode()).hexdigest(), 16) % self.capacity)
    
    def refresh_value(self, key):
        return (self.hash_value(key) + 1) % self.capacity
    
    def search_node(self, key):    # key에 해당하는 버킷 검색
        hash = self.hash_value(key)
        p = self.table[hash]    # 버킷 지정
        
        for i in range(self.capacity):
            if p.stat == Status.EMPTY:    # 비어있으면 값이 없으므로 종료
                break
            elif p.stat == Status.OCCUPIED and p.key == key:
                return p
            else:    # 재해시
                hash = self.refresh_value(hash)
                p = self.table[hash]
                
        return None
    
    def search(self, key):
        p = self.search_node(key)
        if p is not None:
            return p.value
        else:
            return None
        
    def add(self, key, value):
        if self.search(key) is not None:
            return False    # 이미 등록된 키
        
        hash = self.hash_value(key)    # 추가하는 key의 해시값
        p = self.table[hash]
        for i in range(self.capacity):
            if p.stat == Status.EMPTY or p.stat == Status.DELETED:
                self.table[hash] = Bucket(key, value, Status.OCCUPIED)
                return True
            hash = self.refresh_value(hash)    # 재해시
            p = self.table[hash] 
        return False    # table 가득참
    
    def remove(self, key):
        p = self.search_node(key)
        if p is None:
            return False
        else:
            p.set_status(Status.DELETED)
            return True
            
    def dump(self):
        for i in range(self.capacity):
            print(f'{i:2} ', end="")
            if self.table[i].stat == Status.OCCUPIED:
                print(f'{self.table[i].key} ({self.table[i].value})')
            elif self.table[i].stat == Status.EMPTY:
                print("--미등록--")
            elif self.table[i].stat == Status.DELETED:
                print("--삭제 완료--")
            